In [ ]:
!python setup.py install

python3: can't open file '/content/setup.py': [Errno 2] No such file or directory


In [ ]:
!pip install "phe[cli,examples]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2955 sha256=4a25d41635d50c9d5024be7672f46d9ab19bd4eb8ec8e962ae5d0c60be64e23a
  Stored in directory: /root/.cache/pip/wheels/f8/e0/3d/9d0c2020c44a519b9f02ab4fa6d2a4a996c98d79ab2f569fa1
Successfully built sklearn


In [ ]:
!pip install "phe[cli]>1.2"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install phe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 6.1 MB/s eta 0:00:00


In [23]:
import base64
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding


def utf8(s: bytes):
    return str(s, 'utf-8')


private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=4096,
    backend=default_backend()
)
public_key = private_key.public_key()


private_pem = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)

with open('private_key.pem', 'wb') as f:
    f.write(private_pem)

public_pem = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)
with open('public_key.pem', 'wb') as f:
    f.write(public_pem)


with open("private_key.pem", "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,
        backend=default_backend()
    )

with open("public_key.pem", "rb") as key_file:
    public_key = serialization.load_pem_public_key(
        key_file.read(),
        backend=default_backend()
    )


plaintext = b'this is the correct plaintext!'
print(f'plaintext: \033[1;33m{utf8(plaintext)}\033[0m')
encrypted = base64.b64encode(public_key.encrypt(
    plaintext,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
))
print(f'encrypted: \033[1;32m{utf8(encrypted)}\033[0m')


decrypted = private_key.decrypt(
    base64.b64decode(encrypted),
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)
print(f'decrypted: \033[1;31m{utf8(decrypted)}\033[0m')

plaintext: this is the correct plaintext!
encrypted: acXWdGMmRBXF7YdaEBBg7KUHw8kjxZ0djLTDRpVOVcYF1kpb93QemAz2eyRAlarQTRu+k5wihdQYUNIV4pRYNSRPQ8fYNWBbqLwxhOmC38UMYhxlreiqGaKkw95uHdhL+VbnNb2rJMlVO6WuOCBMhkNu4leXHBAvR1eftENTdc3tCqAMEbzEnQHLBGkGPgqNVRUDhwQOu9QIoJKjEDP6wDoNEvGzKAWK27AzqpABGDI5KdJnNF7yWwYWLqNOf1qBW8kEROktoTgqrghGyZmMNN9U/ygl9ecWvTLmzqvjpYcFLnKDoXvMmzviq3WZdVkeg9IB3mSAi6xrwjlJcXPQ4koQvYJoZHwKwb1CcyS1H88k3wyCZZY+swiAh4zyAr6GJh//YHh7AJisvIbGjH1Mmsbuu9+88B0t0Sas72MTd1yCtI6YU7GIkRe26Juhf5RbiihUAlkLvPy5msX5ZlIXyEd7+hM25pGGKgNhYBC8v/bK4V1ON2XQyZBsF1frkFaKQGL6NKy4VlcyjUMkOz0ipPLIPbzh+/AhdfruTmQD5vae5mkiWbSf1iBfO3UTimcVWCPoXM8XfGYN71cBTslsSFkljITkzZ2tHVcSeEeN+/mCdqDIasQCkLTxp/bmYjU4xEkP1j04kB4YT30ROLwvWfYqjSdyqfcdWNydFTRWLAE=
decrypted: this is the correct plaintext!
